In [11]:
import os
import json
import requests
from dotenv import load_dotenv
from groq import Groq


load_dotenv()

True

In [12]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY")
)

In [13]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models"
        }
    ],
    model = "llama3-groq-70b-8192-tool-use-preview"
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial for various reasons:

1. Efficient Processing: Fast language models can process and analyze large amounts of text quickly, which is essential in today's data-driven world where timely insights are critical. They enable researchers and practitioners to rapidly explore, understand, and utilize textual data.

2. Real-time Applications: The speed of these models makes them suitable for real-time applications such as chatbots, virtual assistants, and live translation systems. These applications require immediate responses to provide a seamless user experience.

3. Cost-Effectiveness: Faster models reduce computational costs by requiring less processing time and resources. This makes them more accessible and affordable for a wider range of users and applications.

4. Improved Accuracy: Some fast language models have demonstrated comparable or even better performance than slower models on certain tasks. This suggests that speed does not necessarily compromise 

In [10]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the pharmacologic class of the drug Ibuprofen? and what are some of its common sideeffects? limit the answer to the 10 most common ones."
        }
    ],
    model = "llama3-groq-70b-8192-tool-use-preview"
)

print(chat_completion.choices[0].message.content)

Ibuprofen is classified as a nonsteroidal anti-inflammatory drug (NSAID). Some of the most common side effects of Ibuprofen include:

1. Nausea
2. Vomiting
3. Diarrhea
4. Abdominal pain
5. Headache
6. Dizziness
7. Sleepiness
8. Ringing in the ears
9. Increased sensitivity to light
10. Swelling of the hands and feet


In [14]:
class ReactionCounts:

    JSON = {
        "name": "get_reaction_counts",
        "description": "For a given type of drug, retrieve the types of averse reactions recorded in the drugs database, along with the total count of each type of occurrence.",
        "prarameters": {
            "type": "object", 
            "properties": {
                "drug_type": {
                    "type": "string",
                    "description": "The pharmacologic class of the drug, e.g. nonsteroidal anti-inflammatory drug."
                }
            },
            "required": ["drug_type"]
        }
    }

    def get_reaction_counts(drug_type): 

        drug_name = drug_type.replace(" ", "+")
        url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.pharm_class_epc:"{drug_name}"&count=patient.reaction.reactionmeddrapt.exact'

        response = requests.get(url)
        return response.json()

In [15]:
class TypesReactions:

    JSON = {
        "name": "get_types_reactions",
        "description": "For a given type of drug, retrieve the top ten most frequently occuring types of averse reactions recorded in the drugs database.",
        "prarameters": {
            "type": "object", 
            "properties": {
                "drug_type": {
                    "type": "string",
                    "description": "The pharmacologic class of the drug, e.g. nonsteroidal anti-inflammatory drug."
                }
            },
            "required": ["drug_type"]
        }
    }

    def get_types_reactions(drug_type): 

        drug_name = drug_type.replace(" ", "+")
        url = f'https://api.fda.gov/drug/event.json?search=patient.drug.openfda.pharm_class_epc:"{drug_name}"&count=patient.reaction.reactionmeddrapt.exact&limit=10'

        response = requests.get(url)
        return response.json

In [16]:
def run_conversation(user_prompt): 
    MODEL="llama3-groq-70b-8192-tool-use-preview"

    messages=[
        {
            "role": "system",
            "content": """Answer any queries the user has about medicinal drugs.
            The user will likely use the common name of a medication. Use the provided functions to find more precise information by deriving the pharmacologic class of the drug and then calling the appropriate functions.
            Inform the user that this data is not specific to the mentioned drug, but rather applicabe to the entire class of drugs."""
        },
        {
            "role": "user",
            "content": user_prompt
        }
    ]
    
    tools = [
        {
            "type": "function", 
            "function": ReactionCounts.JSON
        },
        {
            "type": "function",
            "function": TypesReactions.JSON
        }
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=False,
        tools=tools,
        tool_choice="auto"
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls: 
        print(tool_calls)

        available_functions = {
            "get_reaction_counts": ReactionCounts.get_reaction_counts,
            "get_types_reactions": TypesReactions.get_types_reactions
        }

        messages.append(response_message)

        for tool_call in tool_calls: 
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            function_response = function_to_call(
                drug_type=function_args.get("drug_type")
            )

            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool", 
                    "name": function_name,
                    "content": function_response
                }
            )

        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )

        return second_response.choices[0].message.content


In [17]:
print(run_conversation("What are some of the common side effects of ibuprofen?"))

[ChatCompletionMessageToolCall(id='call_kmhd', function=Function(arguments='{}', name='get_types_reactions'), type='function'), ChatCompletionMessageToolCall(id='call_4gqn', function=Function(arguments='{}', name='get_reaction_counts'), type='function')]


AttributeError: 'NoneType' object has no attribute 'replace'